# Amazon Sagemaker for Cohere using Opensearch Sagemaker ML connector Blueprint

---
## Introduction

This notebook discusses the connectivity guide for Amazon Sagemaker Opensearch ML connector specifically for Cohere embed models.

--- 

## Amazon Sagemaker ML Connector for Amazon Opensearch

--- 

In [ ]:
Pre-requisitions

## Pre-requisites:

1. use xyz
2. Install the required packages


--- 

## Getting Started

### Step 0: 

### Step 1: 
Here we will install all the required dependencies to run this notebook

In [3]:
!pip install boto3==1.34.127 -qU --force --quiet --no-warn-conflicts
!pip install numpy==1.26.4 -qU --force --quiet --no-warn-conflicts
!pip install PyYaml -qU --force --quiet --no-warn-conflicts

**Note:** When installing libraries using the pip, you may encounter errors or warnings during the installation process. These are generally not critical and can be safely ignored. However, after installing the libraries, it is recommended to restart the kernel or computing environment you are working in. Restarting the kernel ensures that the newly installed libraries are loaded properly and available for use in your code or workflow.


In [4]:
import boto3
import yaml

### Step 2:
Here, we retrieve the service that are already deployed as a part of the cloudformation template to be used in building the app. The services include
- Opensearch Serverless Collection for the Vector Database

In [5]:
stackname = "sagemakerconnectoropensearch"  # If your stack name differs from "", please provide your stack name here.

In [12]:
cfn = boto3.client('cloudformation')

response = cfn.describe_stack_resources(
    StackName=stackname
)

cfn_outputs = cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']
print(cfn_outputs)
# Get rds secret arn and database endpoint from cloudformation outputs
for output in cfn_outputs:
    if 'CollectionEndpoint' in output['OutputKey']:
        oss_endpoint = output['OutputValue']

    if 'DatabaseEndpoint' in output['OutputKey']:
        db_host = output['OutputValue']

[{'OutputKey': 'CollectionARN', 'OutputValue': 'arn:aws:aoss:us-east-1:709425451936:collection/bipcuj76cdxedctatnqj'}, {'OutputKey': 'IAMUser', 'OutputValue': 'aossadmin'}, {'OutputKey': 'CollectionEndpoint', 'OutputValue': 'https://bipcuj76cdxedctatnqj.us-east-1.aoss.amazonaws.com'}, {'OutputKey': 'DashboardURL', 'OutputValue': 'https://bipcuj76cdxedctatnqj.us-east-1.aoss.amazonaws.com/_dashboards'}]


In [13]:
# Configure AWS credentials and region
region = 'us-east-1'
service = 'es'
credentials = boto3.Session().get_credentials()
#awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)


In [14]:
## Step 2: Define OpenSearch domain endpoint
opensearch_domain = oss_endpoint
print(opensearch_domain)

https://bipcuj76cdxedctatnqj.us-east-1.aoss.amazonaws.com


In [15]:
## Step 2: Define OpenSearch domain endpoint
opensearch_domain = oss_endpoint

## Step 3: Create the IAM role for OpenSearch to access SageMaker
# Note: This step should be done in the AWS Console or using AWS CLI
# Refer to the "Create an IAM role" section in the documentation

## Step 4: Create the OpenSearch ML connector
def create_ml_connector(domain_endpoint, connector_name, role_arn, sagemaker_endpoint):
    url = f'{domain_endpoint}/_plugins/_ml/connectors/_create'
    
    payload = {
        "name": connector_name,
        "description": "Connector for Cohere embed model",
        "version": 1,
        "protocol": "aws_sigv4",
        "credential": {
            "roleArn": role_arn
        },
        "parameters": {
            "region": region,
            "service_name": "sagemaker"
        },
        "actions": [
            {
                "action_type": "predict",
                "method": "POST",
                "headers": {
                    "content-type": "application/json"
                },
                "url": f"https://runtime.sagemaker.{region}.amazonaws.com/endpoints/{sagemaker_endpoint}/invocations",
                "request_body": "{ \"inputs\": \"${parameters.inputs}\" }"
            }
        ]
    }
    
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, auth=awsauth, json=payload, headers=headers)
    
    print(f"Status Code: {response.status_code}")
    print(f"Response: {response.text}")
    
    return response.json()


In [ ]:

# Usage
connector_name = "cohere-embed-sagemaker-connector"
role_arn = "arn:aws:iam::your-account-id:role/your-opensearch-sagemaker-role"
sagemaker_endpoint = "your-sagemaker-endpoint-name"

connector = create_ml_connector(opensearch_domain, connector_name, role_arn, sagemaker_endpoint)

## Step 5: Test the connector

def test_ml_connector(domain_endpoint, connector_id, input_text):
    url = f'{domain_endpoint}/_plugins/_ml/connectors/{connector_id}/_predict'
    
    payload = {
        "parameters": {
            "inputs": input_text
        }
    }
    
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, auth=awsauth, json=payload, headers=headers)
    
    print(f"Status Code: {response.status_code}")
    print(f"Response: {response.text}")
    
    return response.json()

# Usage
connector_id = connector['connector_id']  # Get this from the create_ml_connector response
input_text = "This is a test sentence for embedding."

prediction = test_ml_connector(opensearch_domain, connector_id, input_text)

## Step 6: Use the connector in an OpenSearch index

def create_index_with_ml_connector(domain_endpoint, index_name, connector_id):
    url = f'{domain_endpoint}/{index_name}'
    
    payload = {
        "settings": {
            "index": {
                "knn": True,
                "knn.algo_param.ef_search": 100
            }
        },
        "mappings": {
            "properties": {
                "text_field": {"type": "text"},
                "embedding": {
                    "type": "knn_vector",
                    "dimension": 4096,  # Adjust based on your Cohere model
                    "method": {
                        "name": "hnsw",
                        "space_type": "l2",
                        "engine": "nmslib"
                    }
                }
            }
        },
        "pipelines": {
            "ml_pipeline": {
                "description": "ML pipeline for embedding generation",
                "processors": [
                    {
                        "ml_connector": {
                            "connector_id": connector_id,
                            "input_field": "text_field",
                            "output_field": "embedding"
                        }
                    }
                ]
            }
        }
    }
    
    headers = {"Content-Type": "application/json"}
    response = requests.put(url, auth=awsauth, json=payload, headers=headers)
    
    print(f"Status Code: {response.status_code}")
    print(f"Response: {response.text}")
    
    return response.json()

# Usage
index_name = "cohere-embed-index"
create_index_with_ml_connector(opensearch_domain, index_name, connector_id)

## Step 7: Index a document using the ML pipeline

def index_document(domain_endpoint, index_name, document):
    url = f'{domain_endpoint}/{index_name}/_doc?pipeline=ml_pipeline'
    
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, auth=awsauth, json=document, headers=headers)
    
    print(f"Status Code: {response.status_code}")
    print(f"Response: {response.text}")
    
    return response.json()

# Usage
document = {
    "text_field": "This is a sample document for embedding and indexing."
}

indexed_doc = index_document(opensearch_domain, index_name, document)

## Step 8: Perform a k-NN search

def knn_search(domain_endpoint, index_name, query_text, k=5):
    url = f'{domain_endpoint}/{index_name}/_search'
    
    payload = {
        "size": k,
        "query": {
            "knn": {
                "embedding": {
                    "vector": query_text,
                    "k": k
                }
            }
        }
    }
    
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, auth=awsauth, json=payload, headers=headers)
    
    print(f"Status Code: {response.status_code}")
    print(f"Response: {response.text}")
    
    return response.json()

# Usage
query_text = "Sample query for similarity search"
search_results = knn_search(opensearch_domain, index_name, query_text)

print("Search Results:")
for hit in search_results['hits']['hits']:
    print(f"Score: {hit['_score']}, Text: {hit['_source']['text_field']}")
